In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from menu_functions import get_menu, detect_language, translate_text, change_context
from langchain.schema import Document

def crous():
    path = "C:/Users/busch/OneDrive/Documents/Fac/M2/UE1 - Advanced programming and data visualization/Advanced Programming/projet/environment/"
    load_dotenv(f"{path}.env")
    os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
    
    llm = ChatOpenAI(model="gpt-3.5-turbo")
    question = input("Your question: ")
    history = []
    language = detect_language(question)
    change = False
    
    while question.lower() != "exit":
        # Ajouter la dernière question à l'historique
        history.append({"role": "user", "content": question})
        if change == False:
            menu, restaurant_name = get_menu(question)
            menus = translate_text(menu, language)
            
            # Traitement brut du texte
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
            splits = text_splitter.split_text(menus)

            # Transformer le texte en objets Document
            documents = [Document(page_content=split) for split in splits]

            # Création du VectorStore
            vectorstore = InMemoryVectorStore.from_documents(
                documents=documents, embedding=OpenAIEmbeddings()
            )
            retriever = vectorstore.as_retriever()

            # Prompt
            system_prompt = (
                "You are an assistant for question-answering tasks about the menu of university restaurant. "
                "If you don't specify a specific dish, you should always give today's meal or the nearest one. "
                "If I ask you a question about a restaurant and you don't have the menu of this restaurant, say: 'No context'."
                "If you are unable to provide specific menu information for a restaurant for a date, say : 'No context'."
                "Use the following pieces of retrieved context to answer the question. "
                "Use bullet points when it's necessary. "
                "The menu will be sent to you in markdown text format. After the menu, you find the allergies part. Never show the allergen part in your answer. "
                "You're also an allergy specialist. Allergies are explained in brackets. If there are allergens, you must write a message at the end, for example: ‘Warning! Allergens such as lactose or hazelnuts may be present in this menu'. "
                "When I ask you about allergies, always answer about the allergies of the menu that we talked about before, never all the allergies of all menus."
            )

            # Ajouter la variable context dans le modèle
            prompt_template = PromptTemplate(
                input_variables=[system_prompt, "input"],
                template="{context}\n\nHuman: {input}"
            )
            
            question_answer_chain = create_stuff_documents_chain(llm, prompt_template)
            rag_chain = create_retrieval_chain(retriever, question_answer_chain)

        # Inclure l'historique dans l'entrée
        full_input = {"context": "\n".join([f"{h['role']}: {h['content']}" for h in history]), "input": question}

        results = rag_chain.invoke(full_input)
    
        answer = results.get("answer")#, "No context.")
        translated_answer = translate_text(answer, language)
        print(translated_answer)

        history.append({"role": "assistant", "content": answer})

        # Demander la prochaine question
        question = input("Your question: ")
        change = change_context(question, restaurant_name)
        if change == False:
            history = []

In [3]:
crous()

Your question:  Give me the menu at Gallia


Here is the menu for dinner at Gallia on Monday, January 13, 2025:

STUDENT MENU:
- Beef fricadelle with curry sauce
- Peas
- Macaroni

VEGETARIAN MENU:
- Vegetarian nuggets
- Peas
- Macaroni

Please note the allergens for each dish:
- Beef fricadelle with curry sauce: Contains Gluten and Lactose
- Macaroni: Contains Gluten, Lactose, and Nut

Enjoy your meal!


Your question:  Give me the menu at Esplanade


Here is the menu for Monday, January 13, 2025 at Esplanade for dinner on the 1st floor for students:

Main Course:
- Beef fricadelle with cream sauce
- Green beans
- Roasted potatoes
- Arrabiata pasta gratin

Desserts:
- Paris-Brest
- Lemon tart
- Apple compote with speculoos

Please note the allergens listed for each dish. Enjoy your meal!


Your question:  I am lactose intolerant



No problem! Here are some lactose-free options for you:

- Arrabiata pasta gratin
- Lemon tart (without the crust)
- Apple compote with speculoos

Enjoy your meal!


Your question:  Exit
